In [1]:
import pandas as pd

#load dataset
def xlsx_to_csv_pd():
    data_xls = pd.read_excel('Training data.xlsx', 'Info', index_col=None)
    data_xls.to_csv('Info.csv', encoding='utf-8',index = False)
    data_xls = pd.read_excel('Training data.xlsx', 'TPR', index_col=None)
    data_xls.to_csv('TPR.csv', encoding='utf-8', index = False)

def load_dataset(file):
    #Set = pd.read_csv(file, header = None, names = features)
    Set = pd.read_csv(file, header = None, skiprows=[0])
    return Set

xlsx_to_csv_pd()
Set1 = load_dataset("Info.csv")
Set2 = load_dataset("TPR.csv")

Set2

,0,1,2,3,4,5,6
0,2458948,2016-12-29 13:40:08,36.6,72.0,20.0,116.0,70.0
1,2458948,2016-12-29 14:35:50,36.2,90.0,18.0,111.0,78.0
2,2458948,2016-12-29 19:05:44,36.5,72.0,18.0,123.0,68.0
3,2458948,2016-12-30 06:50:30,36.9,92.0,20.0,127.0,71.0
4,2458948,2016-12-30 14:37:21,36.0,NaN,18.0,NaN,NaN
...,...,...,...,...,...,...,...
8748,14221059,2019-07-26 03:05:15,37.5,NaN,NaN,NaN,NaN
8749,14221059,2019-07-26 05:00:32,37.4,90.0,NaN,NaN,NaN
8750,14221059,2019-07-26 07:26:25,NaN,96.0,NaN,106.0,84.0
8751,14221059,2019-07-26 09:00:54,38.1,92.0,22.0,135.0,84.0


In [2]:
#arrange TPR.csv
from sklearn.decomposition import PCA

Set2.iloc[:, 0:3]

aggregation_functions = {2: 'mean', 3: 'mean', 4: 'mean', 5: 'mean', 6: 'mean'}
newSet2 = Set2.groupby(Set2[0]).aggregate(aggregation_functions)
newSet2
#Set1
   
#X_s2 = np.array(newSet2[:, 0:3])
#reduced_s2 = PCA(n_components=3).fit_transform(X_s2)
#reduced_s2

mergeSet = pd.merge(Set1.iloc[:, 0:3], newSet2, on=0)
#newSet2.count()
mergeSet

,0,1,2_x,2_y,3,4,5,6
0,12654924,1,62,36.547368,111.095238,19.437500,134.352941,82.176471
1,13919017,1,86,36.356250,96.560000,17.176471,136.600000,82.200000
2,13344466,1,81,36.645833,71.772727,18.388889,138.904762,69.619048
3,13225525,2,89,36.940000,85.700000,18.900000,131.600000,77.450000
4,12035675,2,71,36.636364,93.416667,19.136364,119.391304,81.608696
...,...,...,...,...,...,...,...,...
315,3044497,2,39,36.800000,83.772727,18.250000,175.818182,103.590909
316,3507318,2,55,37.137931,96.421053,18.062500,155.842105,94.473684
317,3406405,1,79,36.655556,118.679245,19.145833,123.962264,67.528302
318,3945785,2,77,36.941176,74.631579,19.250000,152.789474,82.473684


In [3]:
#clean data

import re
import numpy as np

pd.options.display.max_columns = None
pd.options.display.max_rows = None

X1 = Set1.iloc[:, 0:-1]
Y1 = Set1.iloc[:, -1]
ant = Set1.iloc[:, 4]
ant = ant.rename("ant")
#print(pd.DataFrame(ant))
bact = Set1.iloc[:, 5]
bact = bact.rename("bact")
como = Set1.iloc[:, 3]
como = como.rename("como")

def arrange_series(ser, ser_name):
    ser = ser.str.replace('\d+', '')
    ser = ser.replace('mg', '', regex=True)
    ser = ser.replace(' g', '', regex=True)
    ser = ser.replace('.g', '', regex=True)
    ser = ser.replace('\t', '', regex=True)
    ser = ser.replace(' ', '', regex=True)
    ser = ser.replace(np.nan, '', regex=True)
    origin_ser = ser

    ser = pd.DataFrame(ser.str.split(',').explode())

    ser = ser.replace('', np.nan, regex=True)
    
    ser.pivot_table(index=[ser_name], aggfunc='size')
    ser = ser.groupby(ser[ser_name]).filter(lambda x : len(x)>3)
    #ant.pivot_table(index=[ser_name], aggfunc='size')
    #ant
    ser = ser.drop_duplicates(subset=None, keep='first', inplace=False)

    ## manually drop na
    #ant = ant.drop(1)
    ser = ser.reset_index(drop=True)
    #print (ser)
    #ant.count()
    ser_features = ser[ser_name].tolist()
    return origin_ser, ser_features

def arrange_dig_series(ser, ser_name):
    ser = ser.replace(np.nan, '', regex=True)
    origin_ser = ser

    ser = pd.DataFrame(ser.str.split(',').explode())

    ser = ser.replace('0', np.nan, regex=True)
    
    ser.pivot_table(index=[ser_name], aggfunc='size')
    ser = ser.groupby(ser[ser_name]).filter(lambda x : len(x)>3)

    ser = ser.drop_duplicates(subset=None, keep='first', inplace=False)

    ser = ser.reset_index(drop=True)

    ser_features = ser[ser_name].tolist()
    return origin_ser, ser_features


origin_ant, ant_features = arrange_series(ant, "ant")
origin_bact, bact_features = arrange_series(bact, "bact")
origin_como, como_features = arrange_dig_series(como, "como")
#origin_ant
#ant_features
origin_bact
bact_features
como_features
Y1.value_counts()

0    240
1     80
Name: 6, dtype: int64

In [4]:
def extract_df(ser, features):
    df= pd.DataFrame()
    for feature in features:
        id = 0
        new_list = []
        for element in ser:
            if feature in element:
                #print ("{} {}/// {}".format(id, element, feature))
                new_list.append(1)
            else:
                new_list.append(0)
            id += 1
    
        df[feature] = new_list
    return df
            
#origin_ant
ant_df = extract_df(origin_ant, ant_features)
bact_df = extract_df(origin_bact, bact_features)
como_df = extract_df(origin_como, como_features)
bact_df

,Escherichiacoli,Pseudomonasaerinosa,Candidaalbicans,Staphylococcusepidermidis,Enterococcusfaecium,Klebsiellapneumoniaessppneumoniae,Proteusmirabilis,Enterococcusfaecalis,Yeast-like
0,1,1,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0
2,0,1,1,1,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0
4,0,0,0,0,1,0,0,0,0
5,0,0,1,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,1,0,0,0
9,0,0,0,0,0,0,0,0,0


In [5]:
from sklearn.decomposition import PCA
X_ant = np.array(ant_df)
reduced_ant = PCA(n_components=12).fit_transform(X_ant)
reduced_ant

X_bact = np.array(bact_df)
#reduced_bact = PCA(n_components=7).fit_transform(X_bact)
reduced_bact = PCA(n_components=3).fit_transform(X_bact)
#reduced_bact
X_como = np.array(como_df)
reduced_como = PCA(n_components=1).fit_transform(X_como)


In [6]:
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, chi2, f_classif 

scaler = StandardScaler()

mergeSet = pd.merge(Set1.iloc[:, 0:3], newSet2, on=0)
#newSet2.count()
mergeSet

mergeSet1 = pd.concat([mergeSet, como_df, pd.DataFrame(reduced_ant), pd.DataFrame(reduced_bact)], axis=1)
#mergeSet1 = pd.concat([mergeSet, como_df, ant_df, bact_df], axis=1)

#newSet2.count()
mergeSet1

,0,1,2_x,2_y,3,4,5,6,1,4,2,0,1,2,3,4,5,6,7,8,9,10,11,0,1,2
0,12654924,1,62,36.547368,111.095238,19.437500,134.352941,82.176471,1,1,0,0.007270,0.475716,0.082784,0.656093,-0.082977,-0.639694,0.463129,0.297031,0.388745,-0.126367,-0.054220,0.167726,0.945355,-0.179535,0.746151
1,13919017,1,86,36.356250,96.560000,17.176471,136.600000,82.200000,1,0,0,-0.444738,-0.037766,0.001496,0.002675,-0.034266,0.007233,-0.002728,0.022985,0.017768,-0.004253,-0.005625,-0.012589,-0.149264,-0.050960,-0.024522
2,13344466,1,81,36.645833,71.772727,18.388889,138.904762,69.619048,0,1,0,0.089402,0.980472,0.191116,-0.528507,-0.013753,0.215537,-0.009015,-0.047343,-0.461315,0.289327,-0.411592,-0.061426,0.009095,0.393149,1.369269
3,13225525,2,89,36.940000,85.700000,18.900000,131.600000,77.450000,1,0,1,-0.444738,-0.037766,0.001496,0.002675,-0.034266,0.007233,-0.002728,0.022985,0.017768,-0.004253,-0.005625,-0.012589,-0.149264,-0.050960,-0.024522
4,12035675,2,71,36.636364,93.416667,19.136364,119.391304,81.608696,1,1,0,-0.163986,0.122105,0.111574,0.243635,0.062398,0.859434,0.024887,-0.018183,0.157227,-0.372943,-0.002503,0.007284,-0.135353,0.105647,0.066156
5,12558022,1,80,36.780000,70.235294,18.933333,137.647059,67.705882,1,0,1,0.245276,1.006694,-0.013443,-0.590181,0.069768,0.553043,-0.512371,0.418853,-0.745321,-0.982481,0.880271,0.960553,-0.108209,0.292859,0.325632
6,14137480,2,73,36.980769,84.594595,17.370370,119.400000,61.600000,1,0,0,-0.380725,-0.072445,-0.302801,0.014395,0.566051,-0.137246,0.080621,-0.172272,-0.104713,-0.029125,0.002498,0.056364,-0.149264,-0.050960,-0.024522
7,12799381,1,93,36.863636,78.560000,19.400000,137.800000,64.850000,1,0,1,-0.444738,-0.037766,0.001496,0.002675,-0.034266,0.007233,-0.002728,0.022985,0.017768,-0.004253,-0.005625,-0.012589,-0.149264,-0.050960,-0.024522
8,13696712,1,58,36.541176,85.166667,17.437500,136.789474,87.842105,0,0,1,0.795322,0.710731,0.671822,-0.481489,0.361210,0.108941,0.299301,-0.475041,1.290001,-0.032717,0.124019,-0.038693,-0.095302,0.813178,-0.206339
9,12621956,2,65,36.642105,102.368421,18.473684,125.888889,75.944444,0,1,0,-0.043384,0.696784,0.277511,-0.342116,-0.379596,-0.414150,0.920427,-0.290528,-0.148072,-0.212309,-0.505863,0.089061,-0.149264,-0.050960,-0.024522


In [7]:
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, chi2, f_classif 

scaler = StandardScaler()

mergeSet = pd.merge(Set1.iloc[:, 0:3], newSet2, on=0)
#newSet2.count()
mergeSet

mergeSet1 = pd.concat([mergeSet, como_df, pd.DataFrame(reduced_ant), pd.DataFrame(reduced_bact)], axis=1)
#mergeSet1 = pd.concat([mergeSet, como_df, ant_df, bact_df], axis=1)

#newSet2.count()
mergeSet1
print(mergeSet1.shape)

X1 = mergeSet1.iloc[:, 1:]
X1 = np.array(X1)

scaler.fit(X1)
X1 = scaler.transform(X1)
pd.DataFrame(X1)

# Select k best features
kb = SelectKBest(f_classif, k=15).fit(X1, Y1)
X = kb.transform(X1)
#f_classif 
pd.DataFrame(X)

(320, 26)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,-1.538968,-0.476026,-0.766678,2.412196,-0.147285,0.367809,1.193416,-0.460566,0.238209,-0.277675,-2.185196,1.124022,-0.248281,0.793275,3.710890
1,-1.538968,0.983407,-1.147601,1.200018,-0.013963,0.369970,1.193416,-0.460566,0.004305,-0.114668,0.024706,0.086979,-0.025756,-0.059540,-0.121957
2,-1.538968,0.679358,-0.570425,-0.867137,0.122782,-0.785308,-0.837931,-0.460566,0.549930,-0.046023,0.736276,-0.179155,-1.884732,-0.290519,6.809895
3,0.649786,1.165836,0.015885,0.294339,-0.310622,-0.066211,1.193416,2.171241,0.004305,-0.114668,0.024706,0.086979,-0.025756,-0.059540,-0.121957
4,0.649786,0.071261,-0.589300,0.937877,-1.034985,0.315672,1.193416,-0.460566,0.321051,0.208810,2.935830,-0.068808,-0.011463,0.034451,0.329021
5,-1.538968,0.618548,-0.303015,-0.995353,0.048160,-0.960990,1.193416,2.171241,-0.038681,0.233473,1.889195,1.585024,4.030870,4.543026,1.619489
6,0.649786,0.192881,0.097143,0.202153,-1.034469,-1.521678,1.193416,-0.460566,-0.871301,1.894234,-0.468831,-0.651913,0.011438,0.266581,-0.121957
7,-1.538968,1.409074,-0.136317,-0.301107,0.057235,-1.223238,1.193416,2.171241,0.004305,-0.114668,0.024706,0.086979,-0.025756,-0.059540,-0.121957
8,-1.538968,-0.719264,-0.779019,0.249861,-0.002722,0.888070,-0.837931,2.171241,1.933148,1.208755,0.372143,-1.797648,0.567897,-0.183002,-1.026203
9,0.649786,-0.293597,-0.577856,1.684416,-0.649472,-0.204463,-0.837931,-0.460566,0.798530,-1.270283,-1.414738,-1.099416,-2.316409,0.421225,-0.121957


In [8]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedShuffleSplit

from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
from sklearn.naive_bayes import CategoricalNB
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV
from statistics import mean

models = ["DecisionTree", "LinearSVC", "naive_byes"]
Y = Y1
def create_classifier(alg):
    if (alg == "DecisionTree"):
        # tune the parameter
        # n_jobs = -1 is to run with all cores
        para = {'max_depth':range(2,20), 'min_samples_leaf':range(2,20)}
        clf = GridSearchCV(DecisionTreeClassifier(), para, cv=5, n_jobs=-1)
    elif (alg == "LinearSVC"):
        para = {'C':[0.01, 0.01, 0.1, 1, 10, 100, 1000, 10000], 'max_iter':[10000]}
        clf = GridSearchCV(svm.LinearSVC(), para, cv=5, n_jobs=-1)
    elif (alg == "naive_byes"):
        # no need of hyper-parameter to tune
        clf = GaussianNB()
    else:
        clf = GaussianNB()
    return clf

#kf = KFold(n_splits=10)
kf = StratifiedShuffleSplit(n_splits=10)
max_score = 0
for model in models:
    clf = create_classifier(model)
    # do 10 folds and caculate score
    score = cross_val_score(clf, X, Y, scoring='f1', cv=kf)
    #score = cross_val_score(clf, X, Y, cv=kf)
    print (model)
    print (score)
    score_avg = mean(score)
    if score_avg > max_score:
        max_score = score_avg
        best_model = model
    print (score_avg)
print (best_model)


clf = create_classifier(best_model)
clf.fit(X, Y)

DecisionTree
[0.30769231 0.46153846 0.61538462 0.76923077 0.46153846 0.46153846
 0.61538462 0.76923077 0.66666667 0.625     ]
0.5753205128205128


/home/daniellin/Desktop/homework/dm_pj1/ENV/lib/python3.8/site-packages/sklearn/svm/_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
/home/daniellin/Desktop/homework/dm_pj1/ENV/lib/python3.8/site-packages/sklearn/svm/_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
/home/daniellin/Desktop/homework/dm_pj1/ENV/lib/python3.8/site-packages/sklearn/svm/_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
/home/daniellin/Desktop/homework/dm_pj1/ENV/lib/python3.8/site-packages/sklearn/svm/_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
/home/daniellin/Desktop/homework/dm_

LinearSVC
[0.57142857 0.36363636 0.66666667 0.77777778 0.66666667 0.93333333
 0.42857143 0.61538462 0.58823529 0.57142857]
0.6183129289011642
naive_byes
[0.46153846 0.58823529 0.625      0.57142857 0.46153846 0.25
 0.46153846 0.46153846 0.375      0.76923077]
0.5025048480930834
LinearSVC


GridSearchCV(cv=5, estimator=LinearSVC(), n_jobs=-1,
             param_grid={'C': [0.01, 0.01, 0.1, 1, 10, 100, 1000, 10000],
                         'max_iter': [10000]})